#### Takes csv output generated from spliceR_python_HLA 
- filter out guides with low score
- find best guides that maximizes number of haplotypes targeted

In [1]:
import numpy as np
import pandas as pd

### HLA-A 

In [2]:
HLA_A_all_guides_df = pd.read_csv("./HLA_A_all_haps_splice_site_BE.csv")
HLA_A_all_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,A*30:04:01:01,1,donor,GACCCCGCACTCACCCGCCC,GACCCCGCACTCACCCGCCCAGG,NGG,CBE,14,0.113088,0.067395
1,A*30:04:01:01,1,donor,CGCACTCACCCGCCCAGGTC,CGCACTCACCCGCCCAGGTCTGG,NGG,CBE,9,0.334910,0.067395
2,A*30:04:01:01,1,donor,GCACTCACCCGCCCAGGTCT,GCACTCACCCGCCCAGGTCTGGG,NGG,CBE,8,0.466535,0.067395
3,A*30:04:01:01,1,donor,CACCCGCCCAGGTCTGGGTC,CACCCGCCCAGGTCTGGGTCAGG,NGG,CBE,3,0.336702,0.067395
4,A*30:04:01:01,1,donor,ACCCGCCCAGGTCTGGGTCA,ACCCGCCCAGGTCTGGGTCAGGG,NGG,CBE,2,0.257234,0.067395
...,...,...,...,...,...,...,...,...,...,...
2011,A*02:17:02:01,7,acceptor,CCCTGGGCACTGTCACTGCC,CCCTGGGCACTGTCACTGCCTGG,NGG,CBE,20,0.044522,0.996357
2012,A*02:17:02:01,7,acceptor,CCTGGGCACTGTCACTGCCT,CCTGGGCACTGTCACTGCCTGGG,NGG,CBE,19,0.059258,0.996357
2013,A*02:17:02:01,7,acceptor,CTGGGCACTGTCACTGCCTG,CTGGGCACTGTCACTGCCTGGGG,NGG,CBE,18,0.085840,0.996357
2014,A*02:17:02:01,7,acceptor,CCCAGGCAGTGACAGTGCCC,CCCAGGCAGTGACAGTGCCCAGG,NGG,ABE,4,0.238371,0.996357


In [3]:
all_A_haps = set(list(HLA_A_all_guides_df['haplotype name']))
print(len(all_A_haps))
all_A_haps

40


{'A*01:01:01:01',
 'A*02:01:01:01',
 'A*02:01:04',
 'A*02:02:01:01',
 'A*02:03:01:01',
 'A*02:05:01:01',
 'A*02:06:01:01',
 'A*02:07:01:01',
 'A*02:11:01:01',
 'A*02:17:02:01',
 'A*03:01:01:01',
 'A*03:02:01:01',
 'A*11:01:01:01',
 'A*11:02:01:01',
 'A*23:01:01:01',
 'A*24:02:01:01',
 'A*24:03:01:01',
 'A*24:07:01:01',
 'A*25:01:01:01',
 'A*26:01:01:01',
 'A*26:02:01',
 'A*26:03:01:01',
 'A*29:01:01:01',
 'A*29:02:01:01',
 'A*30:01:01:01',
 'A*30:02:01:01',
 'A*30:04:01:01',
 'A*31:01:02:01',
 'A*32:01:01:01',
 'A*33:01:01:01',
 'A*33:03:01:01',
 'A*34:01:01:01',
 'A*34:02:01:01',
 'A*36:01:01:01',
 'A*66:01:01:01',
 'A*68:01:01:01',
 'A*68:01:02:01',
 'A*68:02:01:01',
 'A*68:03:01:01',
 'A*74:01:01:01'}

plan: filter out guides that have score <0.7 
make a set of all the guides (to get all unique guides), loop through all of them and for each guide, record how many haplotypes it targets, make a dataframe for each guide 

In [4]:
# filter to get guides with score >0.7
# filter out guides targeting last 3 exons 
num_exons = 8 
score_cutoff = 0.7
last_exons = [num_exons, num_exons-1, num_exons-2]
HLA_A_filtered_guides_df = (HLA_A_all_guides_df[HLA_A_all_guides_df['guide score'] > score_cutoff]).reset_index(drop=True)
pd.set_option('display.max_rows', 60)
HLA_A_filtered_guides_df = HLA_A_filtered_guides_df[~HLA_A_filtered_guides_df['exon number'].isin(last_exons)]

HLA_A_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,A*30:04:01:01,3,acceptor,AGAACCTGGCCCCGACCCCG,AGAACCTGGCCCCGACCCCGCGG,NGG,CBE,6,0.860042,0.564663
1,A*30:04:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029
2,A*30:04:01:01,5,acceptor,AGCTCTGGGAAAAGAGGGGA,AGCTCTGGGAAAAGAGGGGAAGG,NGG,CBE,5,0.839644,0.922587
3,A*24:03:01:01,3,acceptor,AGAACCTGGCCCCGACCCCG,AGAACCTGGCCCCGACCCCGCGG,NGG,CBE,6,0.860042,0.564663
4,A*24:03:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029
...,...,...,...,...,...,...,...,...,...,...
117,A*26:02:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029
118,A*26:02:01,5,acceptor,GGCTCTGGGAAAAGAGGGGA,GGCTCTGGGAAAAGAGGGGAAGG,NGG,CBE,5,0.839644,0.922587
119,A*02:17:02:01,3,acceptor,AGAACCTGGCCCGGACCCCG,AGAACCTGGCCCGGACCCCGCGG,NGG,CBE,6,0.860042,0.564663
120,A*02:17:02:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029


In [5]:
HLA_A_filtered_guides_df.to_csv('HLA_A_filtered_guides_0.7CO_exclLast3Exons.csv', index=False)

In [6]:
# get unique guides 
unique_guides = set(list(HLA_A_filtered_guides_df["protospacer"]))
print(f"with score cutoff {score_cutoff}, HLA-A has {len(unique_guides)} unique guides")
unique_guides
# len(unique_guides)

with score cutoff 0.7, HLA-A has 6 unique guides


{'AGAACCTGGCCCCGACCCCA',
 'AGAACCTGGCCCCGACCCCG',
 'AGAACCTGGCCCGGACCCCG',
 'AGCTCTGGGAAAAGAGGGGA',
 'CCTTACCCCATCTCAGGGTG',
 'GGCTCTGGGAAAAGAGGGGA'}

In [17]:
guide_haplotype_dict = {} # dict mapping guide sequence to dataframe of all info for only that guide 
guide_num_haplotype_dict = {} # dict mapping guide sequence to number of haplotypes targeted by that guide 
guide_haplotype_set_dict = {} # dict mapping guide sequence to list of haplotypes targeted by that guide 
# guide_3a_targets = []
# counter = 0
for g in unique_guides:
    temp_df = (HLA_A_filtered_guides_df[HLA_A_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_haplotype_dict[g]=temp_df
    guide_haplotype_set_dict[g] = set(list(temp_df['haplotype name']))
    guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))
    # if (3 in list((temp_df['exon number']))) and ("acceptor" in list((temp_df['splice site type']))):
    #     counter += 1 
    #     guide_3a_targets.extend(temp_df['haplotype name'])

    
# print(guide_3a_targets)
# print(counter)
# print('unique haps targeted',len(set(guide_3a_targets)))
guide_haplotype_set_dict #dictionary mapping guide seq to set of haplotypes targeted 
guide_num_haplotype_dict # dict mapping guide sequence to number of haplotypes targeted by that guide 
# guide_haplotype_dict

{'AGAACCTGGCCCCGACCCCG': 13,
 'AGAACCTGGCCCGGACCCCG': 19,
 'GGCTCTGGGAAAAGAGGGGA': 31,
 'CCTTACCCCATCTCAGGGTG': 40,
 'AGCTCTGGGAAAAGAGGGGA': 9,
 'AGAACCTGGCCCCGACCCCA': 8}

In [14]:
### VERSION 2: FILTER OUT LAST 3 EXONS AND THEN JUST IGNORE CDNA DISRUPT PCT (since last 3 exons of HLA A encode cytoplasmic tail -> want to disrupt something before that)
# make a score for each guide averaged over all the haplotypes that takes into account guide score, and SPLICE SITE TYPE (this score doesn't have any 'real' interpretation (e.g. edit probability) beyond being useful for ranking)
# rank the unique guides according to score 
# pick guides for the sets going down the ranked list 

edit_site_wt = 0.2 #according to moriarty paper it seems like SD site has ~20% better efficacy for reducing surface expression of target protein compared to SA site edit, but this adjustment weight is pretty arbitrary and can be messed around with 
HLA_A_filtered_guides_df['donor'] = HLA_A_filtered_guides_df['splice site type'].apply(lambda x: 1 if x == "donor" else 0)
HLA_A_filtered_guides_df["overall_score"] = HLA_A_filtered_guides_df["guide score"]+edit_site_wt*HLA_A_filtered_guides_df["donor"] 
HLA_A_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct,donor,overall_score
0,A*30:04:01:01,3,acceptor,AGAACCTGGCCCCGACCCCG,AGAACCTGGCCCCGACCCCGCGG,NGG,CBE,6,0.860042,0.564663,0,0.860042
1,A*30:04:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029,1,0.989564
2,A*30:04:01:01,5,acceptor,AGCTCTGGGAAAAGAGGGGA,AGCTCTGGGAAAAGAGGGGAAGG,NGG,CBE,5,0.839644,0.922587,0,0.839644
3,A*24:03:01:01,3,acceptor,AGAACCTGGCCCCGACCCCG,AGAACCTGGCCCCGACCCCGCGG,NGG,CBE,6,0.860042,0.564663,0,0.860042
4,A*24:03:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029,1,0.989564
...,...,...,...,...,...,...,...,...,...,...,...,...
117,A*26:02:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029,1,0.989564
118,A*26:02:01,5,acceptor,GGCTCTGGGAAAAGAGGGGA,GGCTCTGGGAAAAGAGGGGAAGG,NGG,CBE,5,0.839644,0.922587,0,0.839644
119,A*02:17:02:01,3,acceptor,AGAACCTGGCCCGGACCCCG,AGAACCTGGCCCGGACCCCGCGG,NGG,CBE,6,0.860042,0.564663,0,0.860042
120,A*02:17:02:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.816029,1,0.989564


In [15]:
# getting average score of each unique guide across haplotypes
guide_overall_score_dict = {}
for g in unique_guides:
    temp_df = (HLA_A_filtered_guides_df[HLA_A_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_overall_score_dict[g] = float(temp_df["overall_score"].mean())
    # guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))
    # if (3 in list((temp_df['exon number']))) and ("acceptor" in list((temp_df['splice site type']))):
    #     counter += 1 
    #     guide_3a_targets.extend(temp_df['haplotype name'])
guide_overall_score_dict

{'AGAACCTGGCCCCGACCCCG': 0.8600415602087623,
 'AGAACCTGGCCCGGACCCCG': 0.8600415602087623,
 'GGCTCTGGGAAAAGAGGGGA': 0.8396441390993628,
 'CCTTACCCCATCTCAGGGTG': 0.9895644212821679,
 'AGCTCTGGGAAAAGAGGGGA': 0.8396441390993628,
 'AGAACCTGGCCCCGACCCCA': 0.8600415602087624}

In [18]:
## ranked guide set creation algorithm: 
## iterate through all unique guides, going down the ranked list of unique guides, one (or zero) set is created per iteration—i.e. for each iteration there is a 'base' guide that is used for as many haplotypes as possible 
## for each set/iteration (starting with the best guide), add the 'base' guide to the set, and check if there are 'leftover' haplotypes NOT covered by the base guide. 
##      if no, the set is complete (all haplotypes covered), and we start a new set (move down the list to the next best guide as a 'base' guide)  
##      if yes, go down the list and add the next best guide that covers any previously excluded haplotypes. 
##          repeat this going down the list until all haplotypes are covered, OR there are no more guides that can be added to get a complete set that covers all haplotypes (incomplete sets are NOT added to the final output)

## note that the output sets ARE RANKED from best to worst! i.e. set 0 is the BEST SET 

cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct','num_guides_needed']
HLA_A_final_out_df = pd.DataFrame(columns=cols)

guides_sorted_list = [(k, guide_overall_score_dict[k]) for k in sorted(guide_overall_score_dict, key=guide_overall_score_dict.get, reverse=True)]
guides_sorted_list # list  of tuples of (guide, score)

for set_idx in range(len(guides_sorted_list)):
    
    num_guides_needed = 1 
    cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
    set_df = pd.DataFrame(columns=cols)
    g,score = guides_sorted_list[set_idx]
    guides_of_set = g
    haps_targeted = guide_haplotype_set_dict[g]
    g_dataframe = guide_haplotype_dict[g] 
    g_dataframe['set']=set_idx
    
    # reordering cols 
    g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
    
    set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
    not_included = all_A_haps-haps_targeted # haplotypes NOT targeted by the 'base' guide of this iteration/set 
    leftovers_idx = set_idx+1
    while (len(not_included)>0) and leftovers_idx<len(guides_sorted_list):
        num_guides_needed+= 1 
        g_next,_ = guides_sorted_list[leftovers_idx] # go down the ranked list of guides to the next best guide 
        guides_of_set += (", "+g_next) # add next best guide to set of guides needed for this set 
        haps_targeted = set(guide_haplotype_set_dict[g_next])
        new_covered = haps_targeted.intersection(not_included)
        not_included = not_included-(new_covered)
        g_dataframe = guide_haplotype_dict[g_next] 
        g_dataframe = g_dataframe[g_dataframe['haplotype name'].isin(new_covered)]
        g_dataframe['set']=set_idx
        g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
        set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
        leftovers_idx += 1 
    
    if len(not_included)==0: # check all haplotypes covered 
        set_df["num_guides_needed"] = num_guides_needed
        set_df["guides of set"] = guides_of_set
        HLA_A_final_out_df = pd.concat([HLA_A_final_out_df, set_df], axis=0, ignore_index=True)



/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/378733335.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/378733335.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_A_final_out_df = pd.concat([HLA_A_final_out_df, set_df], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/378733335.py:30: Futur

In [ ]:
cols_f = ['set','num_guides_needed', 'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_A_final_out_df= HLA_A_final_out_df[['set','num_guides_needed',  'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
pd.set_option('display.max_rows', None)
HLA_A_final_out_df

In [20]:
HLA_A_final_out_df.to_csv('HLA_A_final_out.csv', index=False)

### HLA-B

In [22]:
# import ALL guides from raw output csv
HLA_B_all_guides_df = pd.read_csv("./HLA_B_all_haps_splice_site_BE.csv")
pd.set_option('display.max_rows', 60)
display(HLA_B_all_guides_df)

all_B_haps = set(list(HLA_B_all_guides_df['haplotype name']))
print("number of HLA-B haplotypes covered", len(all_B_haps))

# filter to get guides with score >0.7
# filter out guides targeting last 2 exons 
num_exons = 7
score_cutoff = 0.7
last_exons = [num_exons, num_exons-1]
HLA_B_filtered_guides_df = (HLA_B_all_guides_df[HLA_B_all_guides_df['guide score'] > score_cutoff]).reset_index(drop=True)
HLA_B_filtered_guides_df = HLA_B_filtered_guides_df[~HLA_B_filtered_guides_df['exon number'].isin(last_exons)]

HLA_B_filtered_guides_df

# # filter to get guides with score >0.5
# HLA_B_filtered_guides_df = (HLA_B_all_guides_df[HLA_B_all_guides_df['guide score'] > 0.5]).reset_index(drop=True)
# HLA_B_filtered_guides_df

# HLA_B_filtered_guides_df.to_csv('HLA_B_filtered_guides.csv', index=False)

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,B*54:01:01:01,1,donor,CCCTCCCGACCCGCACTCAC,CCCTCCCGACCCGCACTCACCGG,NGG,CBE,20,0.044239,0.067952
1,B*54:01:01:01,1,donor,CGACCCGCACTCACCGGCCC,CGACCCGCACTCACCGGCCCAGG,NGG,CBE,14,0.143533,0.067952
2,B*54:01:01:01,1,donor,GCACTCACCGGCCCAGGTCT,GCACTCACCGGCCCAGGTCTCGG,NGG,CBE,8,0.534763,0.067952
3,B*54:01:01:01,1,donor,CACCGGCCCAGGTCTCGGTC,CACCGGCCCAGGTCTCGGTCAGG,NGG,CBE,3,0.400186,0.067952
4,B*54:01:01:01,1,donor,ACCGGCCCAGGTCTCGGTCA,ACCGGCCCAGGTCTCGGTCAGGG,NGG,CBE,2,0.312800,0.067952
...,...,...,...,...,...,...,...,...,...,...
4208,B*15:27:01:01,6,acceptor,TTCCCACAGGTGGAAAAGGA,TTCCCACAGGTGGAAAAGGAGGG,NGG,ABE,8,0.254955,0.960514
4209,B*15:27:01:01,7,acceptor,CCCTGGGCACTGTCGCTGGC,CCCTGGGCACTGTCGCTGGCTGG,NGG,CBE,20,0.006267,1.000918
4210,B*15:27:01:01,7,acceptor,GGCTGGAGTAGAACAAAAAC,GGCTGGAGTAGAACAAAAACAGG,NGG,CBE,3,0.083324,1.000918
4211,B*15:27:01:01,7,acceptor,TCCAGCCAGCGACAGTGCCC,TCCAGCCAGCGACAGTGCCCAGG,NGG,ABE,4,0.145725,1.000918


number of HLA-B haplotypes covered 76


,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,B*54:01:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330
1,B*54:01:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
2,B*55:01:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330
3,B*55:01:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
4,B*59:01:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330
...,...,...,...,...,...,...,...,...,...,...
147,B*56:02:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
148,B*56:04:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330
149,B*56:04:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
150,B*15:27:01:01,3,acceptor,AGACCCTGGCCCAGGCCCCG,AGACCCTGGCCCAGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330


In [23]:
# get unique guides 
unique_guides = set(list(HLA_B_filtered_guides_df["protospacer"]))
print(f"with score cutoff {score_cutoff}, HLA-B has {len(unique_guides)} unique guides")
unique_guides
# len(unique_guides)

with score cutoff 0.7, HLA-B has 4 unique guides


{'AGACCCTGGCCCAGGCCCCG',
 'AGACCCTGGCCCCGCCCCCG',
 'AGACCCTGGCCCCGGCCCCG',
 'CCTTACCCCATCTCAGGGTG'}

In [24]:
guide_haplotype_dict = {} # dict mapping guide sequence to dataframe of all info for only that guide 
guide_num_haplotype_dict = {} # dict mapping guide sequence to number of haplotypes targeted by that guide 
guide_haplotype_set_dict = {} # dict mapping guide sequence to list of haplotypes targeted by that guide 

for g in unique_guides:
    temp_df = (HLA_B_filtered_guides_df[HLA_B_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_haplotype_dict[g]=temp_df
    guide_haplotype_set_dict[g] = set(list(temp_df['haplotype name']))
    guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))

guide_haplotype_set_dict #dictionary mapping guide seq to set of haplotypes targeted 
guide_num_haplotype_dict 

{'CCTTACCCCATCTCAGGGTG': 76,
 'AGACCCTGGCCCCGGCCCCG': 50,
 'AGACCCTGGCCCAGGCCCCG': 19,
 'AGACCCTGGCCCCGCCCCCG': 7}

In [25]:
### VERSION 2: FILTER OUT LAST 2 EXONS AND THEN JUST IGNORE CDNA DISRUPT PCT 
# make a score for each guide averaged over all the haplotypes that takes into account guide score, and SPLICE SITE TYPE 
# rank the unique guides according to score 
# pick guides for the sets going down the ranked list 

edit_site_wt = 0.2
HLA_B_filtered_guides_df['donor'] = HLA_B_filtered_guides_df['splice site type'].apply(lambda x: 1 if x == "donor" else 0)
HLA_B_filtered_guides_df["overall_score"] = HLA_B_filtered_guides_df["guide score"]+edit_site_wt*HLA_B_filtered_guides_df["donor"] 
HLA_B_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct,donor,overall_score
0,B*54:01:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330,0,0.876060
1,B*54:01:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773,1,0.989564
2,B*55:01:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330,0,0.876060
3,B*55:01:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773,1,0.989564
4,B*59:01:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330,0,0.876060
...,...,...,...,...,...,...,...,...,...,...,...,...
147,B*56:02:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773,1,0.989564
148,B*56:04:01:01,3,acceptor,AGACCCTGGCCCCGGCCCCG,AGACCCTGGCCCCGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330,0,0.876060
149,B*56:04:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773,1,0.989564
150,B*15:27:01:01,3,acceptor,AGACCCTGGCCCAGGCCCCG,AGACCCTGGCCCAGGCCCCGCGG,NGG,CBE,6,0.876060,0.569330,0,0.876060


In [27]:
# getting average score of each unique guide across haplotypes
guide_overall_score_dict = {}
for g in unique_guides:
    temp_df = (HLA_B_filtered_guides_df[HLA_B_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_overall_score_dict[g] = float(temp_df["overall_score"].mean())
    # guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))
    # if (3 in list((temp_df['exon number']))) and ("acceptor" in list((temp_df['splice site type']))):
    #     counter += 1 
    #     guide_3a_targets.extend(temp_df['haplotype name'])
guide_overall_score_dict

{'CCTTACCCCATCTCAGGGTG': 0.9895644212821679,
 'AGACCCTGGCCCCGGCCCCG': 0.8760598034495628,
 'AGACCCTGGCCCAGGCCCCG': 0.8760598034495627,
 'AGACCCTGGCCCCGCCCCCG': 0.8760598034495627}

In [28]:
## guide sets creation 

cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct','num_guides_needed']
HLA_B_final_out_df = pd.DataFrame(columns=cols)

guides_sorted_list = [(k, guide_overall_score_dict[k]) for k in sorted(guide_overall_score_dict, key=guide_overall_score_dict.get, reverse=True)]
guides_sorted_list # list  of tuples of (guide, score)

for set_idx in range(len(guides_sorted_list)):
    
    num_guides_needed = 1 
    cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
    set_df = pd.DataFrame(columns=cols)
    g,score = guides_sorted_list[set_idx]
    guides_of_set = g
    haps_targeted = guide_haplotype_set_dict[g]
    g_dataframe = guide_haplotype_dict[g] 
    g_dataframe['set']=set_idx
    
    # reordering cols 
    g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
    
    set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
    not_included = all_B_haps-haps_targeted
    leftovers_idx = set_idx+1
    while (len(not_included)>0) and leftovers_idx<len(guides_sorted_list):
        
        num_guides_needed+= 1 
        # print(leftovers_idx)
        # print("ni", not_included)
        g_next,_ = guides_sorted_list[leftovers_idx]
        guides_of_set += (", "+g_next)
        haps_targeted = set(guide_haplotype_set_dict[g_next])
        new_covered = haps_targeted.intersection(not_included)
        # print("nc", new_covered)
        not_included = not_included-(new_covered)
        # print("ni2", not_included)
        g_dataframe = guide_haplotype_dict[g_next] 
        g_dataframe = g_dataframe[g_dataframe['haplotype name'].isin(new_covered)]
        g_dataframe['set']=set_idx
        g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
        set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
        leftovers_idx += 1 
    
    if len(not_included)==0: # check all haplotypes covered 
        set_df["num_guides_needed"] = num_guides_needed
        set_df["guides of set"] = guides_of_set
        HLA_B_final_out_df = pd.concat([HLA_B_final_out_df, set_df], axis=0, ignore_index=True)

/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/1533140849.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/1533140849.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_B_final_out_df = pd.concat([HLA_B_final_out_df, set_df], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/1533140849.py:23: Fu

In [29]:
cols_f = ['set','num_guides_needed', 'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_B_final_out_df= HLA_B_final_out_df[['set','num_guides_needed',  'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 60)
HLA_B_final_out_df

,set,num_guides_needed,guides of set,protospacer,haplotype name,exon number,splice site type,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,0,1,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTG,B*54:01:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
1,0,1,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTG,B*55:01:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
2,0,1,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTG,B*59:01:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
3,0,1,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTG,B*44:02:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
4,0,1,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTG,B*81:01:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.822773
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1,3,"AGACCCTGGCCCCGGCCCCG, AGACCCTGGCCCAGGCCCCG, AG...",AGACCCTGGCCCCGCCCCCG,B*14:02:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCGCGG,NGG,CBE,6,0.876060,0.569330
148,1,3,"AGACCCTGGCCCCGGCCCCG, AGACCCTGGCCCAGGCCCCG, AG...",AGACCCTGGCCCCGCCCCCG,B*40:02:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCGCGG,NGG,CBE,6,0.876060,0.569330
149,1,3,"AGACCCTGGCCCCGGCCCCG, AGACCCTGGCCCAGGCCCCG, AG...",AGACCCTGGCCCCGCCCCCG,B*35:05:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCGCGG,NGG,CBE,6,0.876060,0.569330
150,1,3,"AGACCCTGGCCCCGGCCCCG, AGACCCTGGCCCAGGCCCCG, AG...",AGACCCTGGCCCCGCCCCCG,B*40:06:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCGCGG,NGG,CBE,6,0.876060,0.569330


In [30]:
HLA_B_final_out_df.to_csv('HLA_B_final_out.csv', index=False)

### HLA-C

In [44]:
# import ALL guides from raw output csv
HLA_C_all_guides_df = pd.read_csv("./HLA_C_all_haps_splice_site_BE.csv")
HLA_C_all_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,C*07:02:01:01,1,donor,CCTCCCAACCCCGCACTCAC,CCTCCCAACCCCGCACTCACAGG,NGG,CBE,20,0.034682,0.067212
1,C*07:02:01:01,1,donor,AACCCCGCACTCACAGGCCC,AACCCCGCACTCACAGGCCCAGG,NGG,CBE,14,0.115109,0.067212
2,C*07:02:01:01,1,donor,GCACTCACAGGCCCAGGTCT,GCACTCACAGGCCCAGGTCTCGG,NGG,CBE,8,0.471516,0.067212
3,C*07:02:01:01,1,donor,CACAGGCCCAGGTCTCGGTC,CACAGGCCCAGGTCTCGGTCAGG,NGG,CBE,3,0.341184,0.067212
4,C*07:02:01:01,1,donor,ACAGGCCCAGGTCTCGGTCA,ACAGGCCCAGGTCTCGGTCAGGG,NGG,CBE,2,0.261074,0.067212
...,...,...,...,...,...,...,...,...,...,...
1949,C*04:06:01,7,acceptor,CCCTGGGCACTGTTGCTGGC,CCCTGGGCACTGTTGCTGGCTGG,NGG,CBE,20,0.006267,0.996367
1950,C*04:06:01,7,acceptor,CCTGGGCACTGTTGCTGGCT,CCTGGGCACTGTTGCTGGCTGGG,NGG,CBE,19,0.008453,0.996367
1951,C*04:06:01,7,acceptor,CTGGGCACTGTTGCTGGCTG,CTGGGCACTGTTGCTGGCTGGGG,NGG,CBE,18,0.012549,0.996367
1952,C*04:06:01,7,acceptor,CCCAGCCAGCAACAGTGCCC,CCCAGCCAGCAACAGTGCCCAGG,NGG,ABE,4,0.188218,0.996367


In [ ]:
all_C_haps = set(list(HLA_C_all_guides_df['haplotype name']))
# print(len(all_C_haps))
print("number of HLA-C haplotypes covered", len(all_C_haps))
all_C_haps


number of HLA-C haplotypes covered 35


{'C*01:02:01:01',
 'C*02:02:02:01',
 'C*03:02:01',
 'C*03:03:01:01',
 'C*03:03:04',
 'C*03:04:01:01',
 'C*03:04:02:01',
 'C*03:05:01:01',
 'C*03:06:01:01',
 'C*04:01:01:01',
 'C*04:03:01:01',
 'C*04:06:01',
 'C*05:01:01:01',
 'C*06:02:01:01',
 'C*07:01:01:01',
 'C*07:02:01:01',
 'C*07:04:01:01',
 'C*08:01:01:01',
 'C*08:02:01:01',
 'C*08:03:01:01',
 'C*08:04:01:01',
 'C*08:06',
 'C*12:02:02:01',
 'C*12:03:01:01',
 'C*12:04:02:01',
 'C*14:02:01:01',
 'C*14:02:03',
 'C*14:03:01:01',
 'C*15:02:01:01',
 'C*15:05:01:01',
 'C*16:01:01:01',
 'C*16:02:01:01',
 'C*16:04:01:01',
 'C*17:01:01:02',
 'C*18:01:01:01'}

In [46]:
# filter to get guides with score >0.7
# filter out guides targeting last 3 exons 
num_exons = 7
score_cutoff = 0.7
last_exons = [num_exons, num_exons-1]
HLA_C_filtered_guides_df = (HLA_C_all_guides_df[HLA_C_all_guides_df['guide score'] > score_cutoff]).reset_index(drop=True)
pd.set_option('display.max_rows', 60)
HLA_C_filtered_guides_df = HLA_C_filtered_guides_df[~HLA_C_filtered_guides_df['exon number'].isin(last_exons)]

HLA_C_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,C*07:02:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCG,AGACCCTGGCCCCGCCCCCGCGG,NGG,CBE,6,0.876060,0.563124
1,C*07:02:01:01,4,donor,CCTTACCCCAGCTCAGGGTG,CCTTACCCCAGCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
2,C*07:02:01:01,5,acceptor,GGCTCTGGGAAAGGAGGAGA,GGCTCTGGGAAAGGAGGAGAAGG,NGG,CBE,5,0.839644,0.922797
3,C*14:02:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCG,AGACCCTGGCCCCGCCCCCGTGG,NGG,CBE,6,0.876060,0.563124
4,C*14:02:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
...,...,...,...,...,...,...,...,...,...,...
99,C*17:01:01:02,3,acceptor,AGAACCTGGCCCCGCCCCCG,AGAACCTGGCCCCGCCCCCGTGG,NGG,CBE,6,0.860042,0.554066
100,C*17:01:01:02,5,acceptor,GGCTCTGGGAAAGGAGGGGA,GGCTCTGGGAAAGGAGGGGAAGG,NGG,CBE,5,0.839644,0.924039
101,C*04:06:01,3,acceptor,AGACCCTGGCCCCGCCCCCG,AGACCCTGGCCCCGCCCCCGTGG,NGG,CBE,6,0.876060,0.563124
102,C*04:06:01,4,donor,CCTTACTCCATCTCAGGGTG,CCTTACTCCATCTCAGGGTGAGG,NGG,CBE,6,0.766557,0.813806


In [47]:
# get unique guides 
unique_guides = set(list(HLA_C_filtered_guides_df["protospacer"]))
print(f"with score cutoff {score_cutoff}, HLA-C has {len(unique_guides)} unique guides")
unique_guides
# len(unique_guides)

with score cutoff 0.7, HLA-C has 8 unique guides


{'AGAACCTGGCCCCGCCCCCG',
 'AGACCCTGGCCCCGCCCCCG',
 'CCTTACCCCAGCTCAGGGTG',
 'CCTTACCCCATCTCAGGGTG',
 'CCTTACTCCATCTCAGGGTG',
 'GGCCCTGGGAAAGGAGGGGA',
 'GGCTCTGGGAAAGGAGGAGA',
 'GGCTCTGGGAAAGGAGGGGA'}

In [48]:
HLA_C_filtered_guides_df.to_csv('HLA_C_filtered_guides_0.7CO_exclLast2Exons.csv', index=False)

In [49]:
guide_haplotype_dict = {} # dict mapping guide sequence to dataframe of all info for only that guide 
guide_num_haplotype_dict = {} # dict mapping guide sequence to number of haplotypes targeted by that guide 
guide_haplotype_set_dict = {} # dict mapping guide sequence to list of haplotypes targeted by that guide 

for g in unique_guides:
    temp_df = (HLA_C_filtered_guides_df[HLA_C_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_haplotype_dict[g]=temp_df
    guide_haplotype_set_dict[g] = set(list(temp_df['haplotype name']))
    guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))

guide_haplotype_set_dict #dictionary mapping guide seq to set of haplotypes targeted 
guide_num_haplotype_dict 

{'GGCTCTGGGAAAGGAGGGGA': 26,
 'AGAACCTGGCCCCGCCCCCG': 1,
 'GGCCCTGGGAAAGGAGGGGA': 6,
 'GGCTCTGGGAAAGGAGGAGA': 3,
 'AGACCCTGGCCCCGCCCCCG': 34,
 'CCTTACTCCATCTCAGGGTG': 4,
 'CCTTACCCCATCTCAGGGTG': 27,
 'CCTTACCCCAGCTCAGGGTG': 3}

In [50]:
### VERSION 2: FILTER OUT LAST 2 EXONS AND THEN JUST IGNORE CDNA DISRUPT PCT 
# make a score for each guide averaged over all the haplotypes that takes into account guide score, and SPLICE SITE TYPE 
# rank the unique guides according to score 
# pick guides for the sets going down the ranked list 

edit_site_wt = 0.2
HLA_C_filtered_guides_df['donor'] = HLA_C_filtered_guides_df['splice site type'].apply(lambda x: 1 if x == "donor" else 0)
HLA_C_filtered_guides_df["overall_score"] = HLA_C_filtered_guides_df["guide score"]+edit_site_wt*HLA_C_filtered_guides_df["donor"] 
HLA_C_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct,donor,overall_score
0,C*07:02:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCG,AGACCCTGGCCCCGCCCCCGCGG,NGG,CBE,6,0.876060,0.563124,0,0.876060
1,C*07:02:01:01,4,donor,CCTTACCCCAGCTCAGGGTG,CCTTACCCCAGCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806,1,0.989564
2,C*07:02:01:01,5,acceptor,GGCTCTGGGAAAGGAGGAGA,GGCTCTGGGAAAGGAGGAGAAGG,NGG,CBE,5,0.839644,0.922797,0,0.839644
3,C*14:02:01:01,3,acceptor,AGACCCTGGCCCCGCCCCCG,AGACCCTGGCCCCGCCCCCGTGG,NGG,CBE,6,0.876060,0.563124,0,0.876060
4,C*14:02:01:01,4,donor,CCTTACCCCATCTCAGGGTG,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806,1,0.989564
...,...,...,...,...,...,...,...,...,...,...,...,...
99,C*17:01:01:02,3,acceptor,AGAACCTGGCCCCGCCCCCG,AGAACCTGGCCCCGCCCCCGTGG,NGG,CBE,6,0.860042,0.554066,0,0.860042
100,C*17:01:01:02,5,acceptor,GGCTCTGGGAAAGGAGGGGA,GGCTCTGGGAAAGGAGGGGAAGG,NGG,CBE,5,0.839644,0.924039,0,0.839644
101,C*04:06:01,3,acceptor,AGACCCTGGCCCCGCCCCCG,AGACCCTGGCCCCGCCCCCGTGG,NGG,CBE,6,0.876060,0.563124,0,0.876060
102,C*04:06:01,4,donor,CCTTACTCCATCTCAGGGTG,CCTTACTCCATCTCAGGGTGAGG,NGG,CBE,6,0.766557,0.813806,1,0.966557


In [51]:
# getting average score of each unique guide across haplotypes
guide_overall_score_dict = {}
for g in unique_guides:
    temp_df = (HLA_C_filtered_guides_df[HLA_C_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_overall_score_dict[g] = float(temp_df["overall_score"].mean())
    # guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))
    # if (3 in list((temp_df['exon number']))) and ("acceptor" in list((temp_df['splice site type']))):
    #     counter += 1 
    #     guide_3a_targets.extend(temp_df['haplotype name'])
guide_overall_score_dict

{'GGCTCTGGGAAAGGAGGGGA': 0.8396441390000001,
 'AGAACCTGGCCCCGCCCCCG': 0.86004156,
 'GGCCCTGGGAAAGGAGGGGA': 0.719553791,
 'GGCTCTGGGAAAGGAGGAGA': 0.8396441390000001,
 'AGACCCTGGCCCCGCCCCCG': 0.876059803,
 'CCTTACTCCATCTCAGGGTG': 0.9665565359999999,
 'CCTTACCCCATCTCAGGGTG': 0.9895644210000002,
 'CCTTACCCCAGCTCAGGGTG': 0.9895644210000002}

In [52]:
## guide sets creation 

cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct','num_guides_needed']
HLA_C_final_out_df = pd.DataFrame(columns=cols)

guides_sorted_list = [(k, guide_overall_score_dict[k]) for k in sorted(guide_overall_score_dict, key=guide_overall_score_dict.get, reverse=True)]
guides_sorted_list # list  of tuples of (guide, score)

for set_idx in range(len(guides_sorted_list)):
    
    num_guides_needed = 1 
    cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
    set_df = pd.DataFrame(columns=cols)
    g,score = guides_sorted_list[set_idx]
    guides_of_set = g
    haps_targeted = guide_haplotype_set_dict[g]
    g_dataframe = guide_haplotype_dict[g] 
    g_dataframe['set']=set_idx
    
    # reordering cols 
    g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
    
    set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
    not_included = all_C_haps-haps_targeted
    leftovers_idx = set_idx+1
    while (len(not_included)>0) and leftovers_idx<len(guides_sorted_list):
        
        num_guides_needed+= 1 
        # print(leftovers_idx)
        # print("ni", not_included)
        g_next,_ = guides_sorted_list[leftovers_idx]
        guides_of_set += (", "+g_next)
        haps_targeted = set(guide_haplotype_set_dict[g_next])
        new_covered = haps_targeted.intersection(not_included)
        # print("nc", new_covered)
        not_included = not_included-(new_covered)
        # print("ni2", not_included)
        g_dataframe = guide_haplotype_dict[g_next] 
        g_dataframe = g_dataframe[g_dataframe['haplotype name'].isin(new_covered)]
        g_dataframe['set']=set_idx
        g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
        set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
        leftovers_idx += 1 
    
    if len(not_included)==0: # check all haplotypes covered 
        set_df["num_guides_needed"] = num_guides_needed
        set_df["guides of set"] = guides_of_set
        HLA_C_final_out_df = pd.concat([HLA_C_final_out_df, set_df], axis=0, ignore_index=True)

/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/2671990063.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/2671990063.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_C_final_out_df = pd.concat([HLA_C_final_out_df, set_df], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/2671990063.py:23: Fu

In [53]:
cols_f = ['set','num_guides_needed', 'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_C_final_out_df= HLA_C_final_out_df[['set','num_guides_needed',  'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 60)
HLA_C_final_out_df

,set,num_guides_needed,guides of set,protospacer,haplotype name,exon number,splice site type,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,0,5,"CCTTACCCCATCTCAGGGTG, CCTTACCCCAGCTCAGGGTG, CC...",CCTTACCCCATCTCAGGGTG,C*14:02:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
1,0,5,"CCTTACCCCATCTCAGGGTG, CCTTACCCCAGCTCAGGGTG, CC...",CCTTACCCCATCTCAGGGTG,C*14:02:03,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
2,0,5,"CCTTACCCCATCTCAGGGTG, CCTTACCCCAGCTCAGGGTG, CC...",CCTTACCCCATCTCAGGGTG,C*08:04:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
3,0,5,"CCTTACCCCATCTCAGGGTG, CCTTACCCCAGCTCAGGGTG, CC...",CCTTACCCCATCTCAGGGTG,C*08:01:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
4,0,5,"CCTTACCCCATCTCAGGGTG, CCTTACCCCAGCTCAGGGTG, CC...",CCTTACCCCATCTCAGGGTG,C*15:02:01:01,4,donor,CCTTACCCCATCTCAGGGTGAGG,NGG,CBE,6,0.789564,0.813806
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,5,3,"GGCTCTGGGAAAGGAGGGGA, GGCTCTGGGAAAGGAGGAGA, GG...",GGCCCTGGGAAAGGAGGGGA,C*08:01:01:01,5,acceptor,GGCCCTGGGAAAGGAGGGGAAGG,NGG,CBE,5,0.719554,0.922797
206,5,3,"GGCTCTGGGAAAGGAGGGGA, GGCTCTGGGAAAGGAGGAGA, GG...",GGCCCTGGGAAAGGAGGGGA,C*08:03:01:01,5,acceptor,GGCCCTGGGAAAGGAGGGGAAGG,NGG,CBE,5,0.719554,0.922797
207,5,3,"GGCTCTGGGAAAGGAGGGGA, GGCTCTGGGAAAGGAGGAGA, GG...",GGCCCTGGGAAAGGAGGGGA,C*05:01:01:01,5,acceptor,GGCCCTGGGAAAGGAGGGGAAGG,NGG,CBE,5,0.719554,0.922797
208,5,3,"GGCTCTGGGAAAGGAGGGGA, GGCTCTGGGAAAGGAGGAGA, GG...",GGCCCTGGGAAAGGAGGGGA,C*08:02:01:01,5,acceptor,GGCCCTGGGAAAGGAGGGGAAGG,NGG,CBE,5,0.719554,0.922797


In [54]:
HLA_C_final_out_df.to_csv('HLA_C_final_out.csv', index=False)

### HLA-DQB1

In [55]:
# import ALL guides from raw output csv
HLA_DQB1_all_guides_df = pd.read_csv("./HLA_DQB1_all_haps_splice_site_BE_exclude_exons5_6.csv")
HLA_DQB1_all_guides_df 

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,DQB1*03:02:01:01,1,donor,AGCAGTGGCCCTGCACTTAC,AGCAGTGGCCCTGCACTTACCGG,NGG,CBE,20,0.054189,0.139949
1,DQB1*03:02:01:01,1,donor,GCAGTGGCCCTGCACTTACC,GCAGTGGCCCTGCACTTACCGGG,NGG,CBE,19,0.071884,0.139949
2,DQB1*03:02:01:01,1,donor,AGCAGTGGCCCTGCACTTAC,AGCAGTGGCCCTGCACTTACCGG,NGG,ABE,19,0.048073,0.139949
3,DQB1*03:02:01:01,1,donor,GCAGTGGCCCTGCACTTACC,GCAGTGGCCCTGCACTTACCGGG,NGG,ABE,18,0.048073,0.139949
4,DQB1*03:02:01:01,2,acceptor,CTGGTACACGAAATCCTCTG,CTGGTACACGAAATCCTCTGCGG,NGG,CBE,18,0.097482,0.483461
...,...,...,...,...,...,...,...,...,...,...
377,DQB1*06:03:01:01,3,donor,TACGCCACTCCACGGTGATG,TACGCCACTCCACGGTGATGGGG,NGG,ABE,2,0.132133,0.842239
378,DQB1*06:03:01:01,4,acceptor,ACACTCAGTTCTGCTCTTTA,ACACTCAGTTCTGCTCTTTAGGG,NGG,ABE,20,0.033516,0.983461
379,DQB1*06:03:01:01,4,acceptor,CACTCAGTTCTGCTCTTTAG,CACTCAGTTCTGCTCTTTAGGGG,NGG,ABE,19,0.033516,0.983461
380,DQB1*06:03:01:01,4,donor,CACCTTTCTGACTCCTTTGA,CACCTTTCTGACTCCTTTGACGG,NGG,CBE,3,0.282649,0.983461


In [56]:
all_DQB1_haps = set(list(HLA_DQB1_all_guides_df['haplotype name']))
# print(len(all_C_haps))
print("number of HLA-DQB1 haplotypes covered", len(all_DQB1_haps))
all_DQB1_haps

number of HLA-DQB1 haplotypes covered 14


{'DQB1*02:01:01:01',
 'DQB1*03:01:01:01',
 'DQB1*03:02:01:01',
 'DQB1*03:03:02:01',
 'DQB1*04:01:01:01',
 'DQB1*04:02:01:01',
 'DQB1*05:01:01:01',
 'DQB1*05:02:01:01',
 'DQB1*05:03:01:01',
 'DQB1*06:01:01:01',
 'DQB1*06:02:01:01',
 'DQB1*06:03:01:01',
 'DQB1*06:04:01:01',
 'DQB1*06:09:01:01'}

In [60]:
# filter to get guides with score >0.7
# filter out guides targeting last 2 exons 
num_exons = 6
score_cutoff = 0.7
last_exons = [num_exons, num_exons-1]
HLA_DQB1_filtered_guides_df = (HLA_DQB1_all_guides_df[HLA_DQB1_all_guides_df['guide score'] > score_cutoff]).reset_index(drop=True)
pd.set_option('display.max_rows', 60)
HLA_DQB1_filtered_guides_df = HLA_DQB1_filtered_guides_df[~HLA_DQB1_filtered_guides_df['exon number'].isin(last_exons)]

HLA_DQB1_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,DQB1*03:02:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
1,DQB1*02:01:01:01,2,acceptor,AATCCTCTGCGAGGAATCAC,AATCCTCTGCGAGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
2,DQB1*05:02:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACGGG,NGG,CBE,7,0.710725,0.483461
3,DQB1*03:01:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
4,DQB1*05:03:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACGGG,NGG,CBE,7,0.710725,0.469136
5,DQB1*06:04:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
6,DQB1*04:02:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
7,DQB1*06:09:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
8,DQB1*06:02:01:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461
9,DQB1*03:03:02:01,2,acceptor,AATCCTCTGCGGGGAATCAC,AATCCTCTGCGGGGAATCACCGG,NGG,CBE,7,0.710725,0.483461


In [61]:
# get unique guides 
unique_guides = set(list(HLA_DQB1_filtered_guides_df["protospacer"]))
print(f"with score cutoff {score_cutoff}, HLA-DQB1 has {len(unique_guides)} unique guides")
unique_guides
# len(unique_guides)

with score cutoff 0.7, HLA-DQB1 has 2 unique guides


{'AATCCTCTGCGAGGAATCAC', 'AATCCTCTGCGGGGAATCAC'}

In [62]:
guide_haplotype_dict = {} # dict mapping guide sequence to dataframe of all info for only that guide 
guide_num_haplotype_dict = {} # dict mapping guide sequence to number of haplotypes targeted by that guide 
guide_haplotype_set_dict = {} # dict mapping guide sequence to list of haplotypes targeted by that guide 

for g in unique_guides:
    temp_df = (HLA_DQB1_filtered_guides_df[HLA_DQB1_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_haplotype_dict[g]=temp_df
    guide_haplotype_set_dict[g] = set(list(temp_df['haplotype name']))
    guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))

guide_haplotype_set_dict #dictionary mapping guide seq to set of haplotypes targeted 
guide_num_haplotype_dict 

{'AATCCTCTGCGAGGAATCAC': 1, 'AATCCTCTGCGGGGAATCAC': 13}

In [63]:
## since there is obviously only 1 set of 2 guides, there is no need to run the set creation alg 
HLA_DQB1_filtered_guides_df.to_csv('HLA_DQB1_final_out.csv', index=False)

### HLA-DRB1

In [64]:
# import ALL guides from raw output csv
HLA_DRB1_all_guides_df = pd.read_csv("./HLA_DRB1_all_haps_splice_site_BE_exclude_exons5_6.csv")
HLA_DRB1_all_guides_df 

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,DRB1*14:05:01:01,1,donor,CCACAATGTGCACTTACGTC,CCACAATGTGCACTTACGTCTGG,NGG,CBE,17,0.048857,0.126092
1,DRB1*14:05:01:01,1,donor,CCACAATGTGCACTTACGTC,CCACAATGTGCACTTACGTCTGG,NGG,ABE,16,0.045911,0.126092
2,DRB1*14:05:01:01,2,acceptor,GTACTCCAAGAAACGTGCTG,GTACTCCAAGAAACGTGCTGTGG,NGG,CBE,18,0.012222,0.463171
3,DRB1*14:05:01:01,2,acceptor,TACTCCAAGAAACGTGCTGT,TACTCCAAGAAACGTGCTGTGGG,NGG,CBE,17,0.020726,0.463171
4,DRB1*14:05:01:01,2,acceptor,ACTCCAAGAAACGTGCTGTG,ACTCCAAGAAACGTGCTGTGGGG,NGG,CBE,16,0.024067,0.463171
...,...,...,...,...,...,...,...,...,...,...
951,DRB1*14:08,3,donor,TCACTCCATTCCACTGTGAG,TCACTCCATTCCACTGTGAGAGG,NGG,ABE,3,0.144056,0.815231
952,DRB1*14:08,3,donor,CACTCCATTCCACTGTGAGA,CACTCCATTCCACTGTGAGAGGG,NGG,ABE,2,0.075413,0.815231
953,DRB1*14:08,4,acceptor,TCAGACCGTGCTCCTGAGAG,TCAGACCGTGCTCCTGAGAGAGG,NGG,CBE,14,0.116474,0.953808
954,DRB1*14:08,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808


In [65]:
all_DRB1_haps = set(list(HLA_DRB1_all_guides_df['haplotype name']))
# print(len(all_C_haps))
print("number of HLA-DRB1 haplotypes covered", len(all_DRB1_haps))
all_DRB1_haps

number of HLA-DRB1 haplotypes covered 49


{'DRB1*01:01:01:01',
 'DRB1*01:01:02',
 'DRB1*01:02:01:01',
 'DRB1*01:03:01:01',
 'DRB1*03:01:01:01',
 'DRB1*03:02:01:01',
 'DRB1*04:01:01:01',
 'DRB1*04:02:01',
 'DRB1*04:03:01:01',
 'DRB1*04:04:01:01',
 'DRB1*04:05:01:01',
 'DRB1*04:06:01',
 'DRB1*04:07:01:01',
 'DRB1*04:10:01',
 'DRB1*04:11:01:01',
 'DRB1*07:01:01:01',
 'DRB1*08:01:01:01',
 'DRB1*08:02:01:01',
 'DRB1*08:03:02:01',
 'DRB1*08:04:01:01',
 'DRB1*08:04:04',
 'DRB1*09:01:02:01',
 'DRB1*10:01:01:01',
 'DRB1*11:01:01:01',
 'DRB1*11:01:02:01',
 'DRB1*11:02:01:01',
 'DRB1*11:04:01:01',
 'DRB1*11:04:02',
 'DRB1*12:01:01:01',
 'DRB1*12:02:01:01',
 'DRB1*12:02:02:01',
 'DRB1*13:01:01:01',
 'DRB1*13:02:01:01',
 'DRB1*13:03:01:01',
 'DRB1*13:04',
 'DRB1*13:12:01',
 'DRB1*14:01:01:01',
 'DRB1*14:02:01:01',
 'DRB1*14:03:01',
 'DRB1*14:04:01:01',
 'DRB1*14:05:01:01',
 'DRB1*14:06:01',
 'DRB1*14:08',
 'DRB1*15:01:01:01',
 'DRB1*15:02:02:01',
 'DRB1*15:03:01:01',
 'DRB1*15:06:01',
 'DRB1*16:01:01:01',
 'DRB1*16:02:01:01'}

In [72]:
# filter to get guides with score >0.7
# filter out guides targeting last 2 exons 
num_exons = 6
score_cutoff = 0.5
last_exons = [num_exons, num_exons-1]
HLA_DRB1_filtered_guides_df = (HLA_DRB1_all_guides_df[HLA_DRB1_all_guides_df['guide score'] > score_cutoff]).reset_index(drop=True)
pd.set_option('display.max_rows', 60)
HLA_DRB1_filtered_guides_df = HLA_DRB1_filtered_guides_df[~HLA_DRB1_filtered_guides_df['exon number'].isin(last_exons)]

HLA_DRB1_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,DRB1*14:05:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
1,DRB1*14:04:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
2,DRB1*15:02:02:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
3,DRB1*04:05:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
4,DRB1*08:02:01:01,4,acceptor,TGCACCTGAGAGAGGAAGCC,TGCACCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.790670,0.953808
5,DRB1*04:07:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
6,DRB1*15:06:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
7,DRB1*13:02:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
8,DRB1*09:01:02:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
9,DRB1*08:04:01:01,4,acceptor,TGCACCTGAGAGAGGAAGCC,TGCACCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.790670,0.953808


In [73]:
# get unique guides 
unique_guides = set(list(HLA_DRB1_filtered_guides_df["protospacer"]))
print(f"with score cutoff {score_cutoff}, HLA-DQB1 has {len(unique_guides)} unique guides")
unique_guides
# len(unique_guides)

with score cutoff 0.5, HLA-DQB1 has 5 unique guides


{'GTTGAACTAGGAGAAAAACA',
 'TGCACCTGAGAGAGGAAGCC',
 'TGCTCCTGAGAGAGGAAGCA',
 'TGCTCCTGAGAGAGGAAGCC',
 'TGCTCCTGAGAGAGGAATCC'}

In [74]:
guide_haplotype_dict = {} # dict mapping guide sequence to dataframe of all info for only that guide 
guide_num_haplotype_dict = {} # dict mapping guide sequence to number of haplotypes targeted by that guide 
guide_haplotype_set_dict = {} # dict mapping guide sequence to list of haplotypes targeted by that guide 

for g in unique_guides:
    temp_df = (HLA_DRB1_filtered_guides_df[HLA_DRB1_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_haplotype_dict[g]=temp_df
    guide_haplotype_set_dict[g] = set(list(temp_df['haplotype name']))
    guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))

guide_haplotype_set_dict #dictionary mapping guide seq to set of haplotypes targeted 
guide_num_haplotype_dict 

{'TGCACCTGAGAGAGGAAGCC': 5,
 'GTTGAACTAGGAGAAAAACA': 1,
 'TGCTCCTGAGAGAGGAAGCA': 1,
 'TGCTCCTGAGAGAGGAAGCC': 38,
 'TGCTCCTGAGAGAGGAATCC': 4}

In [75]:
covered_DRB1_haps = set(list(HLA_DRB1_filtered_guides_df['haplotype name']))
# print(len(all_C_haps))
print("number of HLA-DRB1 haplotypes covered", len(covered_DRB1_haps))
covered_DRB1_haps

number of HLA-DRB1 haplotypes covered 49


{'DRB1*01:01:01:01',
 'DRB1*01:01:02',
 'DRB1*01:02:01:01',
 'DRB1*01:03:01:01',
 'DRB1*03:01:01:01',
 'DRB1*03:02:01:01',
 'DRB1*04:01:01:01',
 'DRB1*04:02:01',
 'DRB1*04:03:01:01',
 'DRB1*04:04:01:01',
 'DRB1*04:05:01:01',
 'DRB1*04:06:01',
 'DRB1*04:07:01:01',
 'DRB1*04:10:01',
 'DRB1*04:11:01:01',
 'DRB1*07:01:01:01',
 'DRB1*08:01:01:01',
 'DRB1*08:02:01:01',
 'DRB1*08:03:02:01',
 'DRB1*08:04:01:01',
 'DRB1*08:04:04',
 'DRB1*09:01:02:01',
 'DRB1*10:01:01:01',
 'DRB1*11:01:01:01',
 'DRB1*11:01:02:01',
 'DRB1*11:02:01:01',
 'DRB1*11:04:01:01',
 'DRB1*11:04:02',
 'DRB1*12:01:01:01',
 'DRB1*12:02:01:01',
 'DRB1*12:02:02:01',
 'DRB1*13:01:01:01',
 'DRB1*13:02:01:01',
 'DRB1*13:03:01:01',
 'DRB1*13:04',
 'DRB1*13:12:01',
 'DRB1*14:01:01:01',
 'DRB1*14:02:01:01',
 'DRB1*14:03:01',
 'DRB1*14:04:01:01',
 'DRB1*14:05:01:01',
 'DRB1*14:06:01',
 'DRB1*14:08',
 'DRB1*15:01:01:01',
 'DRB1*15:02:02:01',
 'DRB1*15:03:01:01',
 'DRB1*15:06:01',
 'DRB1*16:01:01:01',
 'DRB1*16:02:01:01'}

In [76]:
### VERSION 2: FILTER OUT LAST 2 EXONS AND THEN JUST IGNORE CDNA DISRUPT PCT 
# make a score for each guide averaged over all the haplotypes that takes into account guide score, and SPLICE SITE TYPE 
# rank the unique guides according to score 
# pick guides for the sets going down the ranked list 

edit_site_wt = 0.2
HLA_DRB1_filtered_guides_df['donor'] = HLA_DRB1_filtered_guides_df['splice site type'].apply(lambda x: 1 if x == "donor" else 0)
HLA_DRB1_filtered_guides_df["overall_score"] = HLA_DRB1_filtered_guides_df["guide score"]+edit_site_wt*HLA_DRB1_filtered_guides_df["donor"] 
HLA_DRB1_filtered_guides_df

,haplotype name,exon number,splice site type,protospacer,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct,donor,overall_score
0,DRB1*14:05:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
1,DRB1*14:04:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
2,DRB1*15:02:02:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
3,DRB1*04:05:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
4,DRB1*08:02:01:01,4,acceptor,TGCACCTGAGAGAGGAAGCC,TGCACCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.790670,0.953808,0,0.790670
5,DRB1*04:07:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
6,DRB1*15:06:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
7,DRB1*13:02:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
8,DRB1*09:01:02:01,4,acceptor,TGCTCCTGAGAGAGGAAGCC,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808,0,0.843209
9,DRB1*08:04:01:01,4,acceptor,TGCACCTGAGAGAGGAAGCC,TGCACCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.790670,0.953808,0,0.790670


In [77]:
# getting average score of each unique guide across haplotypes
guide_overall_score_dict = {}
for g in unique_guides:
    temp_df = (HLA_DRB1_filtered_guides_df[HLA_DRB1_filtered_guides_df['protospacer'] == g]).reset_index(drop=True)
    guide_overall_score_dict[g] = float(temp_df["overall_score"].mean())
    # guide_num_haplotype_dict[g]=len(set(list(temp_df['haplotype name'])))
    # if (3 in list((temp_df['exon number']))) and ("acceptor" in list((temp_df['splice site type']))):
    #     counter += 1 
    #     guide_3a_targets.extend(temp_df['haplotype name'])
guide_overall_score_dict

{'TGCACCTGAGAGAGGAAGCC': 0.7906699660395021,
 'GTTGAACTAGGAGAAAAACA': 0.567643368595425,
 'TGCTCCTGAGAGAGGAAGCA': 0.8432090949191209,
 'TGCTCCTGAGAGAGGAAGCC': 0.843209094919121,
 'TGCTCCTGAGAGAGGAATCC': 0.8432090949191209}

In [78]:
## guide sets creation 

cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct','num_guides_needed']
HLA_DRB1_final_out_df = pd.DataFrame(columns=cols)

guides_sorted_list = [(k, guide_overall_score_dict[k]) for k in sorted(guide_overall_score_dict, key=guide_overall_score_dict.get, reverse=True)]
guides_sorted_list # list  of tuples of (guide, score)

for set_idx in range(len(guides_sorted_list)):
    
    num_guides_needed = 1 
    cols = ['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
    set_df = pd.DataFrame(columns=cols)
    g,score = guides_sorted_list[set_idx]
    guides_of_set = g
    haps_targeted = guide_haplotype_set_dict[g]
    g_dataframe = guide_haplotype_dict[g] 
    g_dataframe['set']=set_idx
    
    # reordering cols 
    g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
    
    set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
    not_included = all_DRB1_haps-haps_targeted
    leftovers_idx = set_idx+1
    while (len(not_included)>0) and leftovers_idx<len(guides_sorted_list):
        
        num_guides_needed+= 1 
        # print(leftovers_idx)
        # print("ni", not_included)
        g_next,_ = guides_sorted_list[leftovers_idx]
        guides_of_set += (", "+g_next)
        haps_targeted = set(guide_haplotype_set_dict[g_next])
        new_covered = haps_targeted.intersection(not_included)
        # print("nc", new_covered)
        not_included = not_included-(new_covered)
        # print("ni2", not_included)
        g_dataframe = guide_haplotype_dict[g_next] 
        g_dataframe = g_dataframe[g_dataframe['haplotype name'].isin(new_covered)]
        g_dataframe['set']=set_idx
        g_dataframe = g_dataframe[['set','haplotype name', 'exon number', 'splice site type', 'protospacer', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
        set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
        leftovers_idx += 1 
    
    if len(not_included)==0: # check all haplotypes covered 
        set_df["num_guides_needed"] = num_guides_needed
        set_df["guides of set"] = guides_of_set
        HLA_DRB1_final_out_df = pd.concat([HLA_DRB1_final_out_df, set_df], axis=0, ignore_index=True)

/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/2999888596.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  set_df = pd.concat([set_df, g_dataframe], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/2999888596.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  HLA_DRB1_final_out_df = pd.concat([HLA_DRB1_final_out_df, set_df], axis=0, ignore_index=True)
/var/folders/sm/2bd25dyd1bx5nl6h9b6j4z7c0000gn/T/ipykernel_14335/2999888596.py:

In [79]:
cols_f = ['set','num_guides_needed', 'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']
HLA_DRB1_final_out_df= HLA_DRB1_final_out_df[['set','num_guides_needed',  'guides of set', 'protospacer', 'haplotype name', 'exon number', 'splice site type', 'guide seq with PAM', 'PAM', 'enzyme class', 'CBE/ABE position', 'guide score', 'cDNA disruption pct']]
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 60)
HLA_DRB1_final_out_df

,set,num_guides_needed,guides of set,protospacer,haplotype name,exon number,splice site type,guide seq with PAM,PAM,enzyme class,CBE/ABE position,guide score,cDNA disruption pct
0,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*14:05:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
1,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*14:04:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
2,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*15:02:02:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
3,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*04:05:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
4,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*04:07:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
5,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*15:06:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
6,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*13:02:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
7,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*09:01:02:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
8,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*11:04:01:01,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808
9,0,5,"TGCTCCTGAGAGAGGAAGCC, TGCTCCTGAGAGAGGAAGCA, TG...",TGCTCCTGAGAGAGGAAGCC,DRB1*11:04:02,4,acceptor,TGCTCCTGAGAGAGGAAGCCAGG,NGG,CBE,6,0.843209,0.953808


In [80]:
HLA_DRB1_final_out_df.to_csv('HLA_DRB1_final_out.csv', index=False)